In [ ]:
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms as transforms
import numpy as np
import wandb
# import VisionSparsityProbeExperiments.environments.cifar10_resnet_env as res_net
# import VisionSparsityProbeExperiments.train.train as tr
# import VisionSparsityProbeExperiments
import matplotlib.pyplot as plt
from torchvision import models as models


wandb.login()

wandb: Currently logged in as: idori (idori2). Use `wandb login --relogin` to force relogin


True

In [5]:
transform = transforms.Compose([transforms.Pad((32-28) // 2),
								transforms.ToTensor(),
								transforms.Normalize(tuple([ [0.1307]]), tuple([[0.3081]]))])
#choose your dataset
#analysis_dataset = datasets.FashionMNIST("Vision/data/FashionMNIST",train=False,download=True,transform = transform)
analysis_dataset = datasets.MNIST("Vision/data/MNIST",train=False,download=True,transform = transform)

g = torch.Generator()
#why batch size is 1?
train_loader = torch.utils.data.DataLoader(analysis_dataset,
											  batch_size=1, shuffle=False,
											  generator=g)
model = models.resnet18(pretrained=False, num_classes=10)
model.conv1 = torch.nn.Conv2d(1,model.conv1.weight.shape[0],3,1,1,bias=False)
model.maxpool = nn.MaxPool2d(kernel_size=1, stride=1, padding=0)

model.load_state_dict(torch.load("Trained_models/trained_model_full_model_2.pt"))


<All keys matched successfully>

In [6]:
model.eval()
device = torch.device("cpu")
net_correct = 0
with torch.no_grad():
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        
        output = model(data)
        output = output.cpu()
        target = target.cpu()
        net_pred = output.argmax(dim=1, keepdim=True)
        net_correct += sum(net_pred.reshape(-1) == target).item()
accuracy = net_correct / len(train_loader.dataset)
print(f"accuracy:{accuracy}")


accuracy:0.988
